In [1]:
import os
from openai import OpenAI
from dotenv import load_dotenv
from src.knowledge_base import TranslationKB

In [2]:
load_dotenv()

True

### Make sure API key is working, but not displayed

In [3]:
api_key = os.getenv("OPENAI_API_KEY")
if api_key:
    print("API Key loaded successfully.")
    print(f"API Key Length: {len(api_key)} characters")
    print(f"API Key: {api_key[:4]}")  
else:
    print("Failed to load Open API Key.")


API Key loaded successfully.
API Key Length: 164 characters
API Key: sk-p


### Connect to OpenAI

In [4]:
client = OpenAI(
    base_url="https://api.openai.com/v1",
    api_key=os.getenv("OPENAI_API_KEY"),
)

#### Add OpenAI model 
- Be mindful of cost $$$

In [5]:

# model_name = "gpt-4o"
# Use a cheaper model since I am paying for this.
model_name = "gpt-4o-mini"

### Creating the Agent Prompts

In [6]:
response = client.chat.completions.create(
    model=model_name,
    messages=[
        {"role": "system", "content": "You are a Language Translation assistant. You will translate any English text I provide into French and Italian with corresponding articles "
        ""},
        {"role": "user", "content": "Bicycle!"},
    ],
)
print(response.choices[0].message.content)

French: Le vélo !  
Italian: La bicicletta !


#### Add in Knowledge Base and RAG
- Take a query word and embed as a vector
- Using FAISS, search for closest vector (nearest neighbord) within the index
- Using KB, returns--> closest word and distance.
    - Distance <0.5 is good match and > 0.8 low match

In [7]:
from src.embeddings import (
    build_faiss_index,
    save_faiss_index,
    load_faiss_index,
    search_faiss,
)

/Users/sarahrmclaughlin/Documents/repos/i_spy_an_agent/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/sarahrmclaughlin/Documents/repos/i_spy_an_agent/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
index, words = build_faiss_index()
print(words)

['table', 'chair', 'bank', 'car']


In [9]:
word, dist = search_faiss("baynk", index, words)
print(word, dist)

bank 1.5032156


In [10]:
word, dist = search_faiss("tabel", index, words)
print(word, dist)

table 1.0421065


In [11]:
word, dist = search_faiss("tablee", index, words)
print(word, dist)

table 0.55901253


In [12]:
word, dist = search_faiss("chair", index, words)
print(word, dist)

chair 0.0
